In [2]:
!pip install albumentations

   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
    --------------------------------------- 0.5/38.9 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.6/38.9 MB 4.0 MB/s eta 0:00:10
   -- ------------------------------------- 2.1/38.9 MB 4.1 MB/s eta 0:00:10
   -- ------------------------------------- 2.9/38.9 MB 3.6 MB/s eta 0:00:11
   ---- ----------------------------------- 3.9/38.9 MB 3.9 MB/s eta 0:00:10
   ---- ----------------------------------- 4.7/38.9 MB 3.9 MB/s eta 0:00:09
   ----- ---------------------------------- 5.5/38.9 MB 4.0 MB/s eta 0:00:09
   ------ --------------------------------- 6.6/38.9 MB 4.0 MB/s eta 0:00:09
   ------- -------------------------------- 7.1/38.9 MB 4.0 MB/s eta 0:00:08
   -------- ------------------------------- 8.1/38.9 MB 4.0 MB/s eta 0:00:08
   --------- ------------------------------ 8.9/38.9 MB 4.0 MB/s eta 0:00:08
   --------- ------------------------------ 9.7/38.9 MB 3.9 MB/s eta 0:00:08
   ---

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.2.6 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.2.6 which is incompatible.
streamlit 1.32.0 requires numpy<2,>=1.19.3, but you have numpy 2.2.6 which is incompatible.


In [6]:
from PIL import Image
import numpy as np

# Create a dummy sample image (RGB gradient)
img = np.zeros((256, 256, 3), dtype=np.uint8)
for i in range(256):
    img[:, i] = [i, 255 - i, 128]

Image.fromarray(img).save("sample.jpg")
print("✅ sample.jpg created successfully!")


✅ sample.jpg created successfully!


In [11]:
!pip install tensorflow


  Using cached protobuf-6.33.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.33.0-cp310-abi3-win_amd64.whl (436 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.36.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 6.33.0 which is incompatible.


In [15]:
# -----------------------------------------------------------
# Step 3 (Fixed): Train Model Using Original vs. Augmented Data
# -----------------------------------------------------------

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import albumentations as A
import cv2

# 1️⃣ Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)

# 2️⃣ Define CNN model
def create_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 3️⃣ Train on original dataset
model_original = create_cnn()
print("🚀 Training on Original Dataset...")
history_orig = model_original.fit(x_train, y_train,
                                  validation_data=(x_test, y_test),
                                  epochs=5, batch_size=64, verbose=2)

# 4️⃣ Define Albumentations augmentations (safe for small images)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=20, p=0.4),
    A.RandomScale(scale_limit=0.2, p=0.3)
])

# 5️⃣ Apply augmentations safely (resize back to 32x32)
augmented_images = []
augmented_labels = []

for i in range(20000):  # subset for speed
    img = (x_train[i] * 255).astype(np.uint8)
    aug_img = augment(image=img)['image']
    aug_img = cv2.resize(aug_img, (32, 32))  # ensure consistent shape
    augmented_images.append(aug_img / 255.0)
    augmented_labels.append(y_train[i])

x_train_aug = np.stack(augmented_images)
y_train_aug = np.stack(augmented_labels)

# 6️⃣ Train model on augmented dataset
model_aug = create_cnn()
print("\n🚀 Training on Augmented Dataset...")
history_aug = model_aug.fit(x_train_aug, y_train_aug,
                            validation_data=(x_test, y_test),
                            epochs=5, batch_size=64, verbose=2)

# 7️⃣ Compare results
orig_acc = model_original.evaluate(x_test, y_test, verbose=0)[1]
aug_acc  = model_aug.evaluate(x_test, y_test, verbose=0)[1]

print("\n✅ Results:")
print(f"📊 Accuracy (Original Data): {orig_acc*100:.2f}%")
print(f"📈 Accuracy (Augmented Data): {aug_acc*100:.2f}%")


🚀 Training on Original Dataset...
Epoch 1/5
782/782 - 34s - 44ms/step - accuracy: 0.4591 - loss: 1.5086 - val_accuracy: 0.5721 - val_loss: 1.2341
Epoch 2/5
782/782 - 39s - 50ms/step - accuracy: 0.5986 - loss: 1.1475 - val_accuracy: 0.6174 - val_loss: 1.0959
Epoch 3/5
782/782 - 25s - 33ms/step - accuracy: 0.6461 - loss: 1.0165 - val_accuracy: 0.6444 - val_loss: 1.0335
Epoch 4/5
782/782 - 26s - 33ms/step - accuracy: 0.6775 - loss: 0.9217 - val_accuracy: 0.6610 - val_loss: 0.9764
Epoch 5/5
782/782 - 27s - 34ms/step - accuracy: 0.7058 - loss: 0.8507 - val_accuracy: 0.6761 - val_loss: 0.9271

🚀 Training on Augmented Dataset...
Epoch 1/5
313/313 - 16s - 50ms/step - accuracy: 0.3710 - loss: 1.7451 - val_accuracy: 0.4610 - val_loss: 1.4971
Epoch 2/5
313/313 - 12s - 38ms/step - accuracy: 0.4956 - loss: 1.4103 - val_accuracy: 0.5279 - val_loss: 1.3281
Epoch 3/5
313/313 - 12s - 38ms/step - accuracy: 0.5529 - loss: 1.2728 - val_accuracy: 0.5653 - val_loss: 1.2180
Epoch 4/5
313/313 - 12s - 38ms/ste